In [11]:
import pandas as pd
import psycopg2
import dotenv
import os
dotenv.load_dotenv()

True

```bash
sudo -u postgres psql
# Create a new database
CREATE DATABASE population;
# Create a new user with a password
CREATE USER yzpt WITH PASSWORD 'pwd';
# Grant the user privileges on the database
GRANT ALL PRIVILEGES ON DATABASE population TO yzpt;
# Connect to the new database
\c population;
```

In [4]:
df = pd.read_csv('data/TD_POP1B_2021.csv', sep=';')
df.head()

/tmp/ipykernel_23102/1948031382.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/TD_POP1B_2021.csv', sep=';')


,NIVGEO,CODGEO,LIBGEO,SEXE,AGED100,NB
0,COM,1001,L'Abergement-Clémenciat,1,0,6.789402
1,COM,1001,L'Abergement-Clémenciat,2,0,4.945818
2,COM,1001,L'Abergement-Clémenciat,1,1,7.834433
3,COM,1001,L'Abergement-Clémenciat,2,1,6.814843
4,COM,1001,L'Abergement-Clémenciat,1,2,3.900268


In [31]:
df.shape

(5105484, 6)

In [27]:
# Connect to PostgreSQL
conn = psycopg2.connect(
    host=os.getenv('POSTGRES_HOST'),
    user=os.getenv('POSTGRES_USER'),
    password=os.getenv('POSTGRES_PASSWORD'),
    dbname=os.getenv('POSTGRES_DATABASE'),
    port=os.getenv('POSTGRES_PORT')
)
cur = conn.cursor()


In [28]:
# Create table
cur.execute("""
    CREATE TABLE IF NOT EXISTS age_sex_pyramid (
        NIVGEO TEXT,
        CODGEO TEXT,
        LIBGEO TEXT,
        SEXE INTEGER,
        AGED100 INTEGER,
        NB FLOAT
    )
""")

conn.commit()

In [30]:
import tqdm

# Insert data with progress bar
for _, row in tqdm.tqdm(df.iterrows(), total=len(df), desc="Inserting data"):
    try:
        cur.execute("""
            INSERT INTO age_sex_pyramid (NIVGEO, CODGEO, LIBGEO, SEXE, AGED100, NB)
            VALUES (%s, %s, %s, %s, %s, %s)
        """, tuple(row))
    except Exception as e:
        print(e)
        print(row)

conn.commit()

Inserting data: 100%|██████████| 5105484/5105484 [35:16<00:00, 2411.82it/s]  


In [ ]:
cur.close()


In [ ]:
conn.close()


In [1]:
import pandas as pd
import psycopg2
import os
import dotenv

# Load environment variables
dotenv.load_dotenv()

# Connect to PostgreSQL
conn = psycopg2.connect(
    host=os.getenv('POSTGRES_HOST'),
    user=os.getenv('POSTGRES_USER'),
    password=os.getenv('POSTGRES_PASSWORD'),
    dbname=os.getenv('POSTGRES_DATABASE'),
    port=os.getenv('POSTGRES_PORT')
)

# Query the data and aggregate by CODGEO
query = """
    SELECT CODGEO, SUM(NB) AS total_population_sum
    FROM age_sex_pyramid
    GROUP BY CODGEO
"""

df_population = pd.read_sql(query, conn)

# Optionally, you can change the column names to match the desired format
df_population.columns = ['codegeo', 'total_population_sum']

# Display the resulting DataFrame
print(df_population.head())

# Close the connection
conn.close()


/tmp/ipykernel_173619/2814901086.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_population = pd.read_sql(query, conn)


  codegeo  total_population_sum
0   48181                  60.0
1    2571                1408.0
2   64217                 534.0
3   35250                4542.0
4   55173                  38.0


In [5]:
df_population.to_csv('data/total_population.csv', index=False)

In [4]:
df_population['code'] = df_population['codegeo']